# Data Description

This dataset contains information about companies registered by the Business Integrity Commission (BIC) to collect and dispose of waste materials resulting exclusively from demolition, construction, alterations, or excavations in New York City.

Each record represents an entity approved to operate under the classification of Class 2 C&D Registrants. The information is updated daily and has been publicly available since April 4, 2017

# Columns name  - Dictionary Column


| **Column Name**      | **Description**                                          | **API Field Name**    | **Data Type**        |
|----------------------|----------------------------------------------------------|----------------------|----------------------|
| **CREATED**          | Timestamp of when data is processed for OpenData         | `created`            | Floating Timestamp  |
| **BIC NUMBER**       | Unique BIC file number assigned to the entity            | `bic_number`         | Text                |
| **ACCOUNT NAME**     | Name of the entity                                       | `account_name`       | Text                |
| **TRADE NAME**       | Name under which the entity operates                     | `trade_name`         | Text                |
| **ADDRESS**          | Mailing address of the entity                            | `address`            | Text                |
| **CITY**            | City where the entity is located                         | `city`               | Text                |
| **STATE**            | State where the entity is located                        | `state`              | Text                |
| **POSTCODE**        | Postal code of the entity’s mailing address              | `postcode`           | Text                |
| **PHONE**           | Phone number of the entity                               | `phone`              | Text                |
| **EMAIL**           | Email contact of the entity                              | `email`              | Text                |
| **APPLICATION TYPE** | Type of application filed by the entity                  | `application_type`   | Text                |
| **DISPOSITION DATE** | Date of resolution of the application                   | `disposition_date`   | Text                |
| **EFFECTIVE DATE**   | Date when the registration becomes effective             | `effective_date`     | Text                |
| **EXPIRATION DATE**  | Date when the registration expires                       | `expiration_date`    | Text                |
| **RENEWAL**         | Indicates if the registration is renewable               | `renewal`            | Checkbox            |
| **EXPORT DATE**      | Date when the data was last exported by BIC              | `export_date`        | Floating Timestamp  |
| **LATITUDE**         | Latitude of the mailing address                          | `latitude`           | Text                |
| **LONGITUDE**        | Longitude of the mailing address                         | `longitude`          | Text                |
| **COMMUNITY BOARD**  | Community board based on the mailing address            | `community_board`    | Text                |
| **COUNCIL DISTRICT** | Council district where the entity is located            | `council_district`   | Text                |
| **CENSUS TRACT**     | Census tract associated with the mailing address        | `census_tract`      | Text                |
| **BIN**             | Building Identification Number (BIN)                     | `bin`                | Text                |
| **BBL**             | Borough-Block-Lot (BBL) number                           | `bbl`                | Text                |
| **NTA**             | Neighborhood Tabulation Area                             | `nta`                | Text                |
| **BORO**            | Borough where the entity is located                      | `boro`               | Text                |



In [ ]:
import pandas as pd
import requests
from io import StringIO
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# Base parameters
BASE_URL = "https://data.cityofnewyork.us/resource/cspg-yi7g.csv?$query="
QUERY_TEMPLATE = "SELECT created, account_name, latitude, longitude WHERE created >= '{start_date}' AND created <= '{end_date}' ORDER BY created DESC NULL FIRST"
LIMIT = 5000  # Number of records per request

# Function to fetch data with pagination and date filtering
def fetch_data(offset, start_date, end_date):
    query = QUERY_TEMPLATE.format(start_date=start_date, end_date=end_date)
    url = f"{BASE_URL}{query} LIMIT {LIMIT} OFFSET {offset}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text))
        return df if not df.empty else None
    except Exception as e:
        print(f"Error fetching data at offset {offset}: {e}")
        return None

# Main function to retrieve and consolidate the data
def get_filtered_data(start_date, end_date):
    print(f"Fetching data from {start_date} to {end_date}...")

    # Fetch the first batch to verify if there is data available
    initial_df = fetch_data(0, start_date, end_date)
    if initial_df is None:
        print("No data found within the specified date range.")
        return None

    # List to store results
    all_data = [initial_df]
    total_records = len(initial_df)

    print(f"First batch retrieved: {total_records} records.")

    # Create a list of offsets for pagination
    offsets = list(range(LIMIT, 1000000, LIMIT))  # Up to 1 million records

    # Fetch data in parallel
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda offset: fetch_data(offset, start_date, end_date), offsets))

    # Add only the results that contain data
    for result in results:
        if result is not None:
            all_data.append(result)

    # Combine all data into a single DataFrame
    data = pd.concat(all_data, ignore_index=True)
    print(f"Total records retrieved: {len(data)}")

    return data

# Example usage with specific date range
start_date = "2025-02-01"
end_date = "2025-03-06"
filtered_data = get_filtered_data(start_date, end_date)


Fetching data from 2025-02-01 to 2025-03-06...
First batch retrieved: 5000 records.
Total records retrieved: 23591


In [ ]:
filtered_data

,created,account_name,latitude,longitude
0,2025-03-05T15:34:06.000,N&J HOME IMPROVEMENT CORP.,40.682951,-73.818541
1,2025-03-05T15:34:06.000,ABSOLUTE MASONRY INC.,NaN,NaN
2,2025-03-05T15:34:06.000,BAUDIN'S MASONRY INC.,40.552226,-74.160442
3,2025-03-05T15:34:06.000,ARMA SERVICES CORP.,NaN,NaN
4,2025-03-05T15:34:06.000,"BELLATRON INDUSTRIES, LLC",NaN,NaN
...,...,...,...,...
23586,2025-02-03T15:34:10.000,MARPOL HOME IMPROVEMENT LLC,NaN,NaN
23587,2025-02-03T15:34:10.000,"ALROSE CONSTRUCTION, INC.",NaN,NaN
23588,2025-02-03T15:34:10.000,"YANICK CARTING AND DEMOLITION, INC.",NaN,NaN
23589,2025-02-03T15:34:10.000,CARDELLA TRUCKING CO. INC.,NaN,NaN


In [ ]:
# Convert the 'created' column to datetime format if it is not already
filtered_data['created'] = pd.to_datetime(filtered_data['created'])

In [ ]:
# Filter the data for March 3, 2025
filtered_data_march = filtered_data[(filtered_data['created'].dt.year == 2025) &
                     (filtered_data['created'].dt.month == 3)][['latitude', 'longitude', 'account_name']]

filtered_data_march

,latitude,longitude,account_name
0,40.682951,-73.818541,N&J HOME IMPROVEMENT CORP.
1,NaN,NaN,ABSOLUTE MASONRY INC.
2,40.552226,-74.160442,BAUDIN'S MASONRY INC.
3,NaN,NaN,ARMA SERVICES CORP.
4,NaN,NaN,"BELLATRON INDUSTRIES, LLC"
...,...,...,...
3080,40.707353,-73.800573,JM & SON ROLL OFF CORP
3081,40.643629,-73.971238,JOSE M LOPEZ
3082,NaN,NaN,C SIX TRUCKING INC.
3083,40.719051,-73.949676,BREND RESTORATION LLC


In [ ]:
# Remove rows where latitude or longitude are NaN
filtered_data_march_clean = filtered_data_march.dropna(subset=['latitude', 'longitude'])
filtered_data_march_clean

,latitude,longitude,account_name
0,40.682951,-73.818541,N&J HOME IMPROVEMENT CORP.
2,40.552226,-74.160442,BAUDIN'S MASONRY INC.
5,40.603732,-73.938638,S & S CARTING INC
7,40.868317,-73.848995,KARM CONSTRUCTION INCORPORATED
8,40.636907,-74.010316,PURPLE CAR RENOVATION INC.
...,...,...,...
3079,40.764044,-73.885955,"ABC RESTORATION, INC."
3080,40.707353,-73.800573,JM & SON ROLL OFF CORP
3081,40.643629,-73.971238,JOSE M LOPEZ
3083,40.719051,-73.949676,BREND RESTORATION LLC


In [ ]:
# Save
filtered_data_march_clean.to_csv("demolition.csv", index=False)

In [ ]:
import folium

# Create a map centered on an average location of the data
map_center = [filtered_data_march_clean['latitude'].mean(), filtered_data_march_clean['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Add markers for each point in filtered_data_clean
for _, row in filtered_data_march_clean.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['account_name'],
        tooltip=row['account_name']
    ).add_to(mymap)

# Display the map
mymap
